# 5.6 GPU

本节中我们将讨论如何利用GPU的计算性能进行研究。首先是如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）

In [1]:
!nvidia-smi

Fri Sep 16 07:47:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.06       Driver Version: 452.06       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX250      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   39C    P8    N/A /  N/A |     64MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

在Pytorch中每个数组都有一个设备(device），我们通常将其称为上下文(context)。默认情况下，所有变量和相关的计算都分配给cpu。有时上下文可能是gpu。当我们跨服务器部署作业时事情可能会变得棘手。通过智能地将数组分配给上下文，我们可以最大限度地减少在设备之间传输数据的时间。例如，当在带有GPU的服务器上训练时神经网络时我们通常希望模型参数在gpu上。  
要运行此部分的程序至少需要两个gpu，本节主要为了展示数据如何在不同的设备之间传递。

## 5.6.1 计算设备

我们可以指定用于存储和计算的设备，如CPU和GPU。默认情况下张量实在内存中创建的，然后使用cpu计算它。  
在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第块GPU（从0开始）。 另外，cuda:0和cuda是等价的。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:0')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=0))

In [3]:
# 我们可以查询到可用gpu的数量
torch.cuda.device_count()

1

现在我们定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 5.6.2 张量与GPU

我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

### 5.6.2.1 存储在GPU上

有几种方法可以在GPU上存储张量。 例如，我们可以在创建张量时指定存储设备。接 下来，我们在第一个gpu上创建张量变量X。 在GPU上创建的张量只消耗这个GPU的显存。 我们可以使用nvidia-smi命令查看显存使用情况。 一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设你至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量。

In [7]:
# Y = torch.rand(2, 3, device=try_gpu(1))
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.8582, 0.4929, 0.7631],
        [0.7047, 0.3953, 0.8387]], device='cuda:0')

### 5.6.2.2 复制

如果我们要计算X + Y，我们需要决定在哪里执行这个操作。 例如，我们可以将X传输到第二个GPU并在那里执行操作。 不要简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。 由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算。

In [ ]:
Z = X.cuda(1)
print(X)
print(Z)

现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加。

In [ ]:
Y + Z

假设变量Z已经存在于第二个GPU上。 如果我们还是调用Z.cuda(1)会发生什么？ 它将返回Z，而不会复制并分配新内存。

In [ ]:
Z.cuda(1) is Z

### 5.6.2.3 旁注

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。 但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。 这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收）， 然后才能继续进行更多的操作。 这就是为什么拷贝操作要格外小心。 根据经验，多个小操作比一个大操作糟糕得多。 此外，一次执行几个操作比代码中散布的许多单个操作要好得多（除非你确信自己在做什么）。 如果一个设备必须等待另一个设备才能执行其他操作， 那么这样的操作可能会阻塞。 这有点像排队订购咖啡，而不像通过电话预先订购： 当你到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时， 如果数据不在内存中，框架会首先将其复制到内存中， 这会导致额外的传输开销。 更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

## 5.6.3 神经网络与GPU

类似地，神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上。

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

在接下来的几章中， 我们将看到更多关于如何在GPU上运行模型的例子， 因为它们将变得更加计算密集。

当输入为GPU上的张量时，模型将在同一GPU上计算结果。

In [12]:
net(X)

tensor([[0.7449],
        [0.7449]], device='cuda:0', grad_fn=<AddmmBackward0>)

让我们确认模型参数存储在同一个GPU上。

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

总之，只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型。 在下面的章节中，我们将看到几个这样的例子。

## 5.6.4 小结

·我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用GPU进行过计算。  
·深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。  
·不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将除法全局解释器，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。